In [14]:
import requests

base = 'http://192.168.99.100:5000'

resp = requests.get(base)
resp

<Response [200]>

In [15]:
# GET /environments
resp = requests.get(base + '/environments')
resp

<Response [200]>

In [16]:
resp.json()

[{'name': 'default', 'url': 'http://localhost:8888'}]

In [17]:
# CREATE new environment
d = {'name' : 'new env',
     'url' : 'https://localhost:1234'}
url = base + '/environments'
resp = requests.post(url, json=d)
resp

<Response [200]>

In [18]:
resp.text

''

In [19]:
# Check that new environment is there
resp = requests.get(base + '/environments')
resp.json()

[{'name': 'default', 'url': 'http://localhost:8888'},
 {'name': 'new env', 'url': 'https://localhost:1234'}]

In [20]:
# GET /notebooks/<id 1>
import uuid
i = uuid.UUID(int=1).hex
resp = requests.get(base + '/notebooks/%s' % i)
resp

<Response [200]>

In [21]:
resp.json()

{'description': 'Default DB notebook entry',
 'notebook': {'cells': [],
  'language_info': {},
  'metadata': {'display_name': 'Python 3',
   'language': 'python',
   'name': 'python3'},
  'nbformat': 4,
  'nbformat_minor': 2},
 'title': 'Default DB notebook',
 'uuid': '00000000000000000000000000000001'}

In [22]:
# CREATE new notebook
import nbformat
nb = nbformat.read('API test notebook.ipynb', as_version=nbformat.NO_CONVERT)
# simulate clear output
for item in nb['cells']:
    if item['cell_type'] == 'code':
        item['outputs'] = []
        
js = {'title' : 'uploaded.ipynb',
      'description' : 'API test notebook upload',
      'notebook' : nb}

url = base + '/stages'
resp = requests.post(url, json=js)
resp

<Response [200]>

In [23]:
new_uuid = resp.text
new_uuid

'c6637efb40b948dbb7cd7198e00d36e6'

In [24]:
# GET /notebooks/<new uuid>
import uuid
resp = requests.get(base + '/notebooks/%s' % new_uuid)
resp

<Response [200]>

In [25]:
js = resp.json()
js.keys()

dict_keys(['uuid', 'description', 'title', 'notebook'])

In [26]:
# Write notebook to file
import json
with open(js['title'], 'w') as fp:
    # add nbgallery uuid to notebook metadata
    js['notebook']['metadata']['NBGALLERY_UUID'] = js['uuid']
    s = json.dumps(js['notebook']) # nbformat takes strings to read, not dict objects
    nb = nbformat.reads(s, as_version=nbformat.NO_CONVERT)
    nbformat.write(nb, fp)